In [100]:
%run functions_NoiseStudy.py
%matplotlib 

# global variables
fs = 13
type_plot = ['png', 'svg']

Using matplotlib backend: MacOSX


In [3]:
# actual evaluation time
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

# output folder
save_dir_plots = 'plots/' + today + '_measurement'
if not os.path.exists(save_dir_plots):
    os.makedirs(save_dir_plots)
    
save_dir_res = 'Results/' + today + '_measurement'
if not os.path.exists(save_dir_res):
    os.makedirs(save_dir_res)

In [53]:
# depth profile microsensor
path_ = '/Volumes/HIS-CAMEL/04measurementData/20201127_Noise-vs-resolution-paper/'
file_ms = path_ + 'Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/profiles.txt'

# calibration data 
file_calib = 'Results/20210531_calibration/20210531-1455_calibration_1RoI.hdf5'

# measurement data - images of the optode
file_meas = path_ + 'Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/'

# image cropping to the region of interest (RoI) that should be analyzed
RoI_op = [[(730, 200), (730, 1250), (1290, 1250), (1290, 200)],   # optode1
          [(1560, 200), (1560, 1250), (2100, 1250), (2100, 200)]] # optode2

# Image resolution - determine conversion factor px -> mm 
# user definition using f.ex. imageJ
px2mm, dpi = image_resolution(px=840.6646, dist_mm=30., inch=1.1811)
print('Image resolution: 1cm equals: {:.2f}px, i.e. {:.0f}dpi'.format(px2mm, dpi))

# ................................................................................
# load depth profile(s) of the microsensir for validation of the depth profile
dic_micro = read_microsensor(file_ms=file_ms, encoding='latin-1')

# load measurement data and crop according to the RoI
dint_red, dint_green, dint_ratio = splitImage(path=file_meas, RoI_op=RoI_op)

Image resolution: 1cm equals: 28.02px, i.e. 712dpi


In [57]:
# one example of one optode
inp = 'optode1, set2'
#input('Which optode and which setting shall be visualized? ') # optode1, set2

### Image blur

##### Option-1: Gaussian blur

In [113]:
# preparation lineprofile
surface = (12.9, 12.) # optode 1, optode2
depth_lp = (-4, 4) # depth for 2D line profile 
depth_op =  (-1.5, 1.) # depth for optode excerpt
pos_lp = (7., 3.) # position of the depth profile within the optode
ls_lw = [0] # line with of the drawn depth profile

# kernel size
kernel='gauss'
ls_kernel = [(1,1), (5, 5), (11, 11), (15, 15), (21, 21), (25, 25), (31, 31), (35, 35),
             (41, 41), (45, 45), (51 ,51), (55, 55), (61, 61), (65, 65), (71, 71), 
             (75, 75), (81, 81)]  

# .................................................................
# additional information for visualization
arg = dict({'curve lw': 1.5, 'vmin':-5, 'vmax': 130, 'lw': 0.0, 'offset ms': 0.2,
           'aspect': 4, 'cmap': plt.cm.inferno, 'vmin op': 0, 'vmax op': 100,
           'figsize': (12, 5), 'fontsize': 11, 'marker': ['o', 'd', 'x', '.'],
           'colors': ['slategrey', 'darkorange', 'forestgreen', 'k']})

Option-1.1: Gauss filter applied to ratiometric intensity

In [114]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_lp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v1(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_ratio, px2mm=px2mm, 
                                                ls_lw=ls_lw, blur_type='ratio')), 
                  ls_kernel))

# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_optode = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_ratio, 
                                        surface=surface, depth_min=depth_op[0], 
                                        depth_max=depth_op[1], blur_type='ratio')),
                      ls_kernel))

In [115]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_lp[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimages = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_lp, dO2_optode=dO2_optode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])
    dimages[k] = fig_lp

In [116]:
# save depth profile
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_ratioIntensity-blurred_' + '-'.join([i.strip() for i in inp.split(',')])

for k in dimages.keys():
    name_DP = name_DP_ + '_kernel-' + str(k[0]) + '.'
    for t in type_plot:
        dimages[k].savefig(name_DP + t, dpi=300, transparent=False)

Option-1.2: Gauss filter applied to individual intensity channels red and green

In [110]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_lp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v1(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_red, dint_ch2=dint_green,
                                                px2mm=px2mm, ls_lw=ls_lw, 
                                                blur_type='single')), ls_kernel))

# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_optode = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_red, 
                                        dint_ch2=dint_green, surface=surface, 
                                        depth_min=depth_op[0], depth_max=depth_op[1], 
                                        blur_type='single')), ls_kernel))

In [111]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_lp[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimages = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_lp, dO2_optode=dO2_optode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])
    dimages[k] = fig_lp

In [112]:
# save depth profile
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_individualChannels-blurred_' + '-'.join([i.strip() for i in inp.split(',')])

for k in dimages.keys():
    name_DP = name_DP_ + '_kernel-' + str(k[0]) + '.'
    for t in type_plot:
        dimages[k].savefig(name_DP + t, dpi=300, transparent=False)

Option-1.3: Gauss filter applied to normalized ratiometric intensity

In [120]:
%run functions_NoiseStudy.py

In [135]:
# based on O2_lineprofile_compare_v1 blur normalized ratiometric intensity
dint = dint_ratio

# ........................................................................
# preparation
o = inp.split(',')[0]
s = inp.split(',')[1].strip()

# convert from px to mm
dimages = px2mm_conversion(df=pd.DataFrame(dint[o][s]), px2mm=px2mm, 
                           surface=surface[int(o[-1])-1])

# crop to image width of interest
ddlp = dict(map(lambda lw_: (lw_, line_profile_v1(df=dimages, lw=lw_, lp=lp[0])), ls_lw))

# load calibration
calib_info = load_calibration_para_v1(path_calib=path_calib)
para = calib_info[o][s]

# update determination of O2 concentration for line profile
# based on O2concentration_lp
# ........................................................................
